<a href="https://colab.research.google.com/github/rjsdn2308/machine-learning-project/blob/main/%EC%97%B0%EB%A0%B9%EB%B3%84_%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 필요한 라이브러리 설치 및 로드하기

In [ ]:
!pip install pandas numpy scikit-learn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


#2. 데이터 전처리
- 영화 개요, 징르, 키워드 등 핵심 정보를 추출
- JSON 형식 컬럼을 파싱하여 텍스트 토큰화

In [ ]:
# 데이터 로딩
movies = pd.read_csv('/content/drive/MyDrive/ml/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/ml/tmdb_5000_credits.csv')

# release_date → 연도로 변환
movies['release_year'] = pd.to_datetime(movies['release_date'], errors='coerce').dt.year

# 필요 컬럼만 선택
movies = movies[['id','title','overview','genres','keywords','release_year','popularity','vote_average']]

# JSON 문자열 파싱 함수
import ast
def parse_list_column(text):
    try:
        items = ast.literal_eval(text)
        return [ x['name'].replace(' ', '') for x in items ]
    except:
        return []

# genres, keywords 전처리
movies['genres_list']   = movies['genres'].apply(parse_list_column)
movies['keywords_list'] = movies['keywords'].apply(parse_list_column)


<ipython-input-9-2cde93acc190>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres_list']   = movies['genres'].apply(parse_list_column)


#3. 콘텐츠 기반 특징 결합


In [ ]:
# 텍스트 피쳐 결합
def combine_features(row):
    tokens = row['genres_list'] + row['keywords_list']
    # overview는 단어 추출량이 많으므로 길이 제한
    overview = row['overview'] if isinstance(row['overview'], str) else ''
    return ' '.join(tokens) + ' ' + overview

movies['meta'] = movies.apply(combine_features, axis=1)
# 장르, 키워드, 줄거리 등을 하나의 메타 문자열로 결합하여 텍스트 피처 생성

#4. TF-IDF 벡터화 및 코사인 유사도

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(movies['meta'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
# 영화 간 유사도 행렬 생성
# 유사도 기반 추천에 사용

#5. 유사 영화 추천 시스템

In [ ]:
# 타이틀 → 인덱스 매핑
title_to_idx = pd.Series(movies.index, index=movies['title']).drop_duplicates()

def recommend_similar(title, top_n=10): #사용자가 좋아하는 영화와 유사한 다른 영화를 추천
    idx = title_to_idx[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1: top_n+1]
    rec_indices = [ i for i, _ in sim_scores ]
    return movies.iloc[rec_indices][['title','release_year','popularity','vote_average']]


#6. 연령 기반 영화 추천 함수

In [1]:
import datetime

CURRENT_YEAR = datetime.datetime.now().year  # 예: 2025

def recommend_by_age(age, top_n=10): # 사용자의 출생 연도를 기준으로 특정 시기의 영화를 추천
    birth_year = CURRENT_YEAR - age
    # 예: 사용자가 25세라면, 1990~2000년대 영화 추천
    start = birth_year + 5   # 5~15세 시기
    end   = birth_year + 15
    mask = (movies['release_year'] >= start) & (movies['release_year'] <= end)

    subset = movies[mask].sort_values('popularity', ascending=False)
    return subset.head(top_n)[['title','release_year','popularity','vote_average']]

def recommend_age_content(age, favorite_title, top_n=5): # 사용자의 연령대를 고려한 후, 해당 범위 내 유사 영화를 추천
    """연령 필터 후, 특정 영화 기반 유사도 추천"""
    # 연령별 범위 인덱스
    yr_start = (CURRENT_YEAR - age) + 5
    yr_end   = (CURRENT_YEAR - age) + 15
    valid_idx = movies[(movies['release_year']>=yr_start)&(movies['release_year']<=yr_end)].index
    # 해당 연령대 풀에서 유사도 추출
    idx0 = title_to_idx[favorite_title]
    scores = [(i, cosine_sim[idx0][i]) for i in valid_idx]
    scores = sorted(scores, key=lambda x: x[1], reverse=True)[1: top_n+1]
    recs = [i for i,_ in scores]
    return movies.iloc[recs][['title','release_year','popularity','vote_average']]
